# Generate decision tree and visualize it graphically

## Reload on each run, since we’re editing the decision_tree library

In [1]:
%load_ext autoreload
%autoreload 2

## Import libraries

In [3]:
from decision_tree import *
import networkx as nx
import graphviz
from IPython.display import SVG
from IPython.core.display import display

## Sample data

In [4]:
# sample data with a bit of repetition
witnessData = {'wit1': ['a', 'b', 'c', 'a', 'd', 'e'],
               'wit2': ['a', 'e', 'c', 'd'],
               'wit3': ['a', 'd', 'b']}

## Stopword list

Currently we calculate whether either of the tokens in a skip bigram is a stopword, but we don’t use the information. Since we’re incorporating information about the uniqueness of the skipgrams, we already have an alternative way to find those that contain types that occur frequently.

In [5]:
# fake stoplist, to ensure that we can identify stopwords and process them last
stoplist = {'a', 'c'}  # set

## Use bitarrays to keep track of which witness tokens have already been placed

We also use the bitarrays to count the number of placed tokens, which might be use as part of the scoring process.

In [6]:
# bitArray_dict is used to keep track of which witness tokens have already been processed
bitArray_dict = {k: bitarray(len(witnessData[k])) for k in witnessData}  # create a bitarray the length of each witness
for ba in bitArray_dict.values():  # initialize bitarrays to all 0 values
    ba.setall(0)

## Common skipgram table

All skipgrams, with all locations where they occur

In [7]:
# csTable: dictionary, in which
#   key: two-item tuple representing skipgram normalized token values (token[0], token[1])
#   value: list of three-item tuples records all locations where the key occurs: (siglum, offset[0], offset[1])
#     In Real Life:
#       values will include the t values corresponding to the normalized token values
#       use a named tuple or dataclass (https://realpython.com/python-data-classes/)
# In this test sample, we find all skip bigrams; in Real Life we would specify parameters for:
#   size of skipgram (bi, tri-, etc.; here bi-)
#   size of window (maximum distance between first and last members of skipgram; here the full witness length)
#   maximum size of skip between members of skipgram (here constrained only by size of window)
csTable = collections.defaultdict(list)
for key, value in witnessData.items():  # key is siglum, value is list of normalized token readings
    # in Real Life the value would also include a non-normalized t property
    for first in range(len(value)):  # all first items in bigram
        for second in range(first + 1, len(value)):  # pair with all following items
            csTable[(value[first], value[second])].append((key, first, second))

## Shape skipgram table into df and break out features

Features are used to short table of (remaining) skipgrams by priority, that is, to determine what to process next

In [8]:
# convert to series before df since list lengths vary
csSeries = pd.Series(csTable)

In [9]:
# convert series to dataframe, flatten MultiIndex, label columns
csDf = pd.DataFrame(csSeries).reset_index()
csDf.columns = ["first", "second", "locations"]

### Prioritize skipgrams for processing

Three features, in order:

1. How many witnesses does a skipgram occur in (depth)? Integer; higher is better
1. Does any norm value in a skipgram occur more than once in any witness? Boolean; False is better
1. How many times does a skipgram occur in the documents overall? Integer; higher is better, since we’ve already filtered out those where high frequency of a skipgram is accompanied by repetition within a witness

In [10]:
# count witnesses for each skipgram (depth of block) and check for uniqueness of skipgram in all witnesses
#   extract sigla inside set comprehension to remove duplicates, then count
csDf["local_witnesses"] = csDf["locations"].map(lambda x: [location[0] for location in x])
csDf["unique_witnesses"] = csDf["local_witnesses"].map(lambda x: set(x))
csDf["local_witnessCount"] = csDf["local_witnesses"].str.len()
csDf["unique_witnessCount"] = csDf["unique_witnesses"].str.len()
csDf["witness_uniqueness"] = csDf["local_witnessCount"] == csDf["unique_witnessCount"]
scale = pd.Series([100, -1, 10]) # TODO: check this for polarity
csDf["priority"] = pd.np.dot(csDf[["unique_witnessCount", "witness_uniqueness", "local_witnessCount"]], scale)

In [11]:
# are both tokens are stopwords? (if so, we’ll process them last)
# NB: currently we ignore stopwords
csDf["stopwords"] = csDf[["first", "second"]].T.isin(stoplist).all()

In [12]:
# sort and update row numbers, so that we can traverse the skipgrams as follows
csDf.sort_values(by=["priority"], ascending=False, inplace=True)
csDf.reset_index(inplace=True, drop=True)  # update row numbers

### Check the df

In [13]:
csDf

,first,second,locations,local_witnesses,unique_witnesses,local_witnessCount,unique_witnessCount,witness_uniqueness,priority,stopwords
0,a,d,"[(wit1, 0, 4), (wit1, 3, 4), (wit2, 0, 3), (wi...","[wit1, wit1, wit2, wit3]","{wit1, wit3, wit2}",4,3,False,340,False
1,a,e,"[(wit1, 0, 5), (wit1, 3, 5), (wit2, 0, 1)]","[wit1, wit1, wit2]","{wit1, wit2}",3,2,False,230,False
2,a,b,"[(wit1, 0, 1), (wit3, 0, 2)]","[wit1, wit3]","{wit1, wit3}",2,2,True,219,False
3,a,c,"[(wit1, 0, 2), (wit2, 0, 2)]","[wit1, wit2]","{wit1, wit2}",2,2,True,219,True
4,c,d,"[(wit1, 2, 4), (wit2, 2, 3)]","[wit1, wit2]","{wit1, wit2}",2,2,True,219,False
5,a,a,"[(wit1, 0, 3)]",[wit1],{wit1},1,1,True,109,True
6,b,c,"[(wit1, 1, 2)]",[wit1],{wit1},1,1,True,109,False
7,b,a,"[(wit1, 1, 3)]",[wit1],{wit1},1,1,True,109,False
8,b,d,"[(wit1, 1, 4)]",[wit1],{wit1},1,1,True,109,False
9,b,e,"[(wit1, 1, 5)]",[wit1],{wit1},1,1,True,109,False


## Create decision tree

## Build decision tree

Currently we create the root and then expand all branches down three levels. In Real Life:

1. Evaluate scores at each stage to decide what to expand and what not to expand.
1. Navigate levels with function, instead of in one nested `for` structure

In [14]:
# root of decision tree inherits empty toList, bitArray_dict with 0 values, and complete, sorted df
dtRoot = dtNode([Node("#start"), Node("#end")], "[none]", bitArray_dict, csDf)

In [15]:
# process root
parent: dtNode = dtRoot # node to expand
current, remainder = step(csDf) # current is rows to add, remainder is ... well ... what’s left
for i in range(len(current)):
    expand_dtNode(parent, current.iloc[i, :], pd.concat([current.drop(i, axis=0), remainder]))  # expands in place, adds children
    print(parent)
for child in parent.children:
    print("\nOne level down")
    print_alignment_table(child, witnessData, True)  # before expanding
    print(child)
    print_score(child)
    print("Placed skipgram:", child.skipgram)
    print("Percentage of witness tokens placed:", print_placed_witness_tokens(child))
    current_c, remainder_c = step(child.df)
    for j in range(len(current_c)):
        expand_dtNode(child, current_c.iloc[j, :], pd.concat([current_c.drop(j, axis=0), remainder_c]))
        for grandchild in child.children:
            print("\nTwo levels down")
            print_alignment_table(grandchild, witnessData, True)  # before expanding
            print(grandchild)
            print_score(grandchild)
            print("Placed skipgram:", grandchild.skipgram)
            print("Percentage of witness tokens placed:", print_placed_witness_tokens(grandchild))
            current_d, remainder_d = step(grandchild.df)
            for k in range(len(current_d)):
                expand_dtNode(grandchild, current_d.iloc[k, :], pd.concat([current_d.drop(k, axis=0), remainder_d]))
                for greatgrandchild in grandchild.children:
                    print("\nThree levels down")
                    print_alignment_table(greatgrandchild, witnessData, True)
                    print(greatgrandchild)
                    print_score(greatgrandchild)
                    print("Placed skipgram:", greatgrandchild.skipgram)
                    print("Percentage of witness tokens placed:", print_placed_witness_tokens(greatgrandchild))


There are 2 choices at this level
[#start, #end]

One level down
+------+--------+------+------+------+
| wit1 | #start | a(0) | d(4) | #end |
| wit2 | #start | a(0) | d(3) | #end |
| wit3 | #start | a(0) | d(1) | #end |
+------+--------+------+------+------+
[#start, a, d, #end]
Score (witness tokens / toList length):  3.0
Placed skipgram: ad
Percentage of witness tokens placed: 0.46153846153846156

There are 2 choices at this level

Two levels down
+------+--------+------+------+------+------+------+
| wit1 | #start | a(0) |      | d(4) | e(5) | #end |
| wit2 | #start | a(0) | e(1) | d(3) |      | #end |
| wit3 | #start | a(0) |      | d(1) |      | #end |
+------+--------+------+------+------+------+------+
[#start, a, e, d, e, #end]
Score (witness tokens / toList length):  2.0
Placed skipgram: ae
Percentage of witness tokens placed: 0.6153846153846154

There are 1 choices at this level

Three levels down
+------+--------+------+------+------+------+------+
| wit1 | #start | a(0) |

### Visualize decision tree

In [377]:
G = nx.DiGraph()

# create functions to add nodes and edges to networkx DiGraph()
# node ids are unique integers because labels may repeat across branches

def create_adder_n(g: nx.DiGraph): # specify graph and initialize counter when creating adder
    counter = 0 # closure
    def add_node(n: dtNode): # specify only the node when adding it
        nonlocal counter # ensure closure
        counter += 1
        g.add_node(counter, label=n.skipgram, c=counter)
        return counter # to refer to new node
    return add_node

def create_adder_e(g: nx.DiGraph): # specify graph when creating adder
    def add_edge(u: int, v: int): # networkx in and out nodes for edge
        g.add_edge(u, v)
    return add_edge

a_n = create_adder_n(G) # create adder for nodes
a_e = create_adder_e(G) # create adder for edges

In [378]:
r = a_n(dtRoot)
for a in dtRoot.children:
    new_node = a_n(a)
    new_edge = a_e(1, G.node[new_node]["c"])
    for b in a.children:
        new_node = a_n(b)
        new_edge = a_e(a, new_node)

In [379]:
G.nodes().data()

NodeDataView({1: {'label': '[none]', 'c': 1}, 2: {'label': 'ad', 'c': 2}, 3: {'label': 'ae', 'c': 3}, [#start, a, d, #end]: {}, 4: {'label': 'ae', 'c': 4}, 5: {'label': 'ad', 'c': 5}, 6: {'label': 'ae', 'c': 6}, [#start, a, d, #end]: {}, 7: {'label': 'ae', 'c': 7}})

In [380]:
G.edges()

OutEdgeView([(1, 2), (1, 5), ([#start, a, d, #end], 3), ([#start, a, d, #end], 4), ([#start, a, d, #end], 6), ([#start, a, d, #end], 7)])